# Multi-Instrument Timbre Transfer using Omnizart and DDSP

This Notebook does illustrate how we combined those two pieces of software to
create a multi-instrument timbre transfer system.

This is a local version of the code that only runs on the CPU. Hence, it takes
some time to run. Using a GPU lead to issues with Tensorflow on our local
machine. In parallel we also ran this code on the Scitas Izar cluster where we
did have those issues but unfortunately the synthesizer software to resynthesize
the midis generated bby Omnizart is not installable on the cluster. Thus, the
workflow was rather unpractical involving downloading the midis, synthesizing
them and then uploading those audio files back to the cluster.

In [1]:
from omni_transcribe import transcribe, synth
from ddsp_timbre_transfer import timbre_transfer
from utils import combine_wavs
import scipy.io.wavfile as wave

# Disable GPU
import tensorflow as tf
gpus = tf.config.list_physical_devices(device_type = 'GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)
tf.config.set_visible_devices([], 'GPU')

Use Omnizart to perform source separation and transcribe the different
instruments into separate midi files.

In [ ]:
# Transcribe to midi
filename = "test.wav"
midis = transcribe(filename)

At this point, the midis must be resynthesized using a software synthesizer
because DDSP takes wav files as input and not midis directly.

In [ ]:
# Synthesize midi to wav files
for midi in midis:
    synth(midi)

At this stage, the timbre transfer enabled through DDSP is applied to each of
the separated instruments.

In [ ]:
wavs = [midi[2:].replace(".mid","")+"_synth.wav" for midi in midis]
valid_wavs = [wav for wav in wavs if wave.read(wav)[1].size > 0]

# {"Violin", "Flute", "Flute2", "Trumpet", "Tenor_Saxophone"}
all_instruments = ["Violin", "Flute", "Trumpet", "Tenor_Saxophone"]
length_instr = len(all_instruments)
instruments = [all_instruments[i % length_instr] for i in range(len(valid_wavs))]

pairs = zip(valid_wavs, instruments)

# Perform timbre transfer
results = [timbre_transfer(wav, instrument) for wav, instrument in pairs]

And finally those different audio files, each with a transferred timbre, are
combined back into a single audio file, resulting in a timbre-transferred multi
instrument track.

In [ ]:
# Combine wav files into one
out_path = "result.wav"
combine_wavs(results, out_path)